# 1-import Data


In [5]:
import pandas as pd

In [6]:

cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID','PULocationID',
        'DOLocationID', 'payment_type', 'total_amount']

taxi_jan_2019 = pd.read_parquet('yellow_tripdata_2019-01.parquet', columns=cols)

# To add new data in the future
taxi_db = pd.concat([taxi_jan_2019])

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

# Data exploration

In [7]:
taxi_db.head(100_000)

NameError: name 'taxi_db' is not defined

In [ ]:
taxi_db.shape

In [ ]:
taxi_db.hist(figsize=(20,10), bins=60)

In [ ]:
taxi_db['RatecodeID'].value_counts()


In [ ]:
taxi_db.reset_index().plot(kind='scatter', y='total_amount', x='index', figsize=(10,5))

In [8]:
taxi_db[taxi_db['total_amount']<1000].reset_index().plot(kind='scatter', y='total_amount', x='index', figsize=(10,6))

NameError: name 'taxi_db' is not defined

In [ ]:
print(taxi_db[taxi_db['total_amount']<0].shape)
taxi_db[taxi_db['total_amount']<0].reset_index().plot(figsize=(10,5) ,kind='scatter', y='total_amount', x='index')

In [ ]:
taxi_db[taxi_db['total_amount']<0].head()

In [ ]:
taxi_db[taxi_db['total_amount']<0]['payment_type'].value_counts()

In [ ]:
taxi_db[taxi_db['total_amount']==0].shape
taxi_db[taxi_db['total_amount']==0].head()

In [9]:
taxi_db_prepared = taxi_db.copy()

NameError: name 'taxi_db' is not defined

In [ ]:
taxi_db_prepared = taxi_db_prepared[(taxi_db_prepared['total_amount']>=0)&(taxi_db_prepared['total_amount']<2000)]
taxi_db_prepared.dtypes

In [ ]:
taxi_db_prepared['transiction_year'] = taxi_db_prepared['tpep_pickup_datetime'].dt.year
taxi_db_prepared['transiction_month'] = taxi_db_prepared['tpep_pickup_datetime'].dt.month
taxi_db_prepared['transiction_day'] = taxi_db_prepared['tpep_pickup_datetime'].dt.day
taxi_db_prepared['transiction_hour'] = taxi_db_prepared['tpep_pickup_datetime'].dt.hour
taxi_db_prepared['transiction_date'] = taxi_db_prepared['tpep_pickup_datetime'].dt.normalize()


In [ ]:
taxi_db_prepared.head()

In [10]:
taxi_db_prepared.sample(5000).hist(figsize=(20,10), bins = 60)

NameError: name 'taxi_db_prepared' is not defined

In [ ]:
taxi_db_prepared = taxi_db_prepared[taxi_db_prepared['transiction_year'] == 2019]
taxi_db_prepared = taxi_db_prepared[taxi_db_prepared['transiction_month'] == 1]

In [ ]:
taxi_db_prepared.head()


In [ ]:
from sklearn.preprocessing import LabelEncoder

categioral_columns = ['PULocationID', 'transiction_date','transiction_month','transiction_day','transiction_hour']
numerical_columns = ['trip_distance']
target = 'total_amount'
df_encoded= taxi_db_prepared[categioral_columns + numerical_columns+[target]].copy()

for col in categioral_columns:
  le= LabelEncoder()
  df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))

In [11]:
taxi_grouped_by_region = df_encoded.groupby(categioral_columns).mean().reset_index()
taxi_grouped_by_region['transiction_count'] = df_encoded.groupby(categioral_columns).count().reset_index()['total_amount']
print(taxi_grouped_by_region.shape)
taxi_grouped_by_region.head()

NameError: name 'df_encoded' is not defined

In [ ]:
taxi_grouped_by_region['total_amount'].hist(bins=100, figsize=(10,5))


In [ ]:
taxi_grouped_by_region['trip_distance'].hist(bins=100, figsize=(10,5))

In [ ]:
from sklearn.model_selection import train_test_split
x = df_encoded[categioral_columns + numerical_columns]
y = df_encoded[[target]]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)


In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=10)
tree.fit(x_train,y_train)

In [ ]:
model_at_hand = tree

y_pred = model_at_hand.predict(x_test)

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

print('mean_absolute_error',mean_absolute_error(y_test,y_pred))
print('mean_squared_error',mean_squared_error(y_test,y_pred))
print('sqrt',sqrt(mean_squared_error(y_test,y_pred)))
print('r2_score',r2_score(y_test, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    "max_depth" :[5,10,15,20],
    "min_samples_leaf" : [1,5,10]
}

grid = GridSearchCV(DecitionTreeRegressor(), param_grid=params, cv=3)